<a href="https://colab.research.google.com/github/playbyte/ia/blob/main/neurona_PERCEPTRON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

############################################
* Filename:    neurona_PERCEPTRON.ipynb
* Versión:     1.1  (18-11-2025)
* Autor:       Jose M Morales
* Web:  <a href="https://playbyte.es/articulos/indice-ia.html">PLAYBYTE: Redes Neuronales Artificiales</a>
* Licencia: <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Share-Alike 4.0</a>              
############################################

<center>

#  **Neuronas Artificiales: El PERCEPTRON simple**

</center>

>>## **1 - Definición de Perceptrón Simple**
>>## **2 - Implementacion en  PYTHON** (+ Numpy)
>>## **3 - Funciones Auxiliares**
>>## **4 - Ejemplos:**

>>> - Representación de puertas lógicas AND, OR,...
>>> - Ejemplo genérico
>>> - Pesos y alturas de hombres y mujeres
>>> - Concesión de préstamos en un banco

<hr>
<hr>

## <b>1 - Definición de Perceptrón Simple</b>
El perceptrón es la red neuronal más básica que existe de aprendizaje supervisado.

Dispone de $n$ entradas analógicas y una salida $ \widehat{y} $ binaria

<img src="https://playbyte.es/articulos/ia/20231120_neurona-artificial2/perceptron_esquema.png">

Se representa matematicamente: $ \widehat{y}= $
$ step( \sum\limits_{i=1}^n w_i\cdot \textbf{x}_i + b) $

> $ \textbf{x}_i $ : cada una de las n entradas  
 b : (bias)  entrada  'dummy', sirve de ajuste y es independiente de las entradas $ \textbf{x}_i  $  
 $  \widehat{y} $ : salida obtenida

<br>

#### 1.1 <u>Función de activación</u>:
Construimos nuestro perceptron usando una funcion de activación del tipo 'step' (tresshold):

La función de agregación z, es la suma ponderada de las entradas:
 z=  $ \sum\limits_{i=1}^n w_i\cdot \textbf{x}_i + b $

y la función de activación $h(z)= step(z)$ proporciona la salida del Perceptrón $ \widehat{y} $

<img src="https://playbyte.es/articulos/ia/20231120_neurona-artificial2/f_salto.png">

<br>

#### 1.2 <u>Propagación</u>:

Si consideramos $ w_0=b , x_0=1 $ podemos usar esta expresión más genérica:

>$ \widehat{y}= $
$ step( \sum\limits_{i=0}^n w_i\cdot \textbf{x}_i)  $

<br>


#### 1.3 <u>Regla de Aprendizaje</u>:


Ajuste de los pesos: $ w_i → w_i + \alpha· Error ·x_i $

- $  Error = y - \widehat{y} $
- $  \alpha  $ es el coeficiente de aprendizaje (learning rate -lr-)

- $  x_i  $ es la entrada

<hr>


## <b>2 - Implementación en PYTHON</b>
Creamos la clase perceptrón, este código nos sirve de base para ejecutar varios ejemplos

In [ ]:
#####################################################################
#  Filename:    perceptron_class.py
#  Versión:     1.1  (18-11-2025)
#  Autor:       Jose M Morales
#  Web:         https://playbyte.es/articulos/ia/20231212_neurona-artificial3/perceptron2.html
#  Licencia:    Creative Commons Share-Alike 4.0
#               https://creativecommons.org/licenses/by-nc-sa/4.0/
#
#  Descripción: Clase en Python que implementa un PERCEPTRÓN simple
#               con 'n' entradas analogicas, una salida digital ŷ
#                          _________
#                          |       |
#                    x1 -->|w1     |       ⌈ 1, si  w1x1+w2x2 +... > b
#                    x2 -->|w2   _ |--> ŷ= |
#                    ..    |   _|  |       ⌊ 0, si  w1x1+w2x2 +... ≤ b
#                    xn -->|wn     |
#                          |___w0__|
#                              |
#                              b
#####################################################################*/
import numpy as np  # Requiere libreria numpy


class Perceptron:
    """
    <<< Perceptrón simple de n entradas como clasificador lineal >>>

    Entradas:           X: [x1, x2, ..., xn]
                        activación: 'step', 'relu', 'sigmoid', 'tanh'
                        lr: tasa de aprendizaje

    Salidas:            Pesos: w0 (bias), w1, w2, ..., wn
                        Error cuadrático medio (loss)
                        Exactitud (accuracy)
                        Precisión final (score)

    Ejemplo de uso para un perceptron de 2 entradas:

        mi_perceptron = Perceptron(n_inputs=2, activacion="step", lr=0.01)
        mi_perceptron.fit(training_data, learn_rate, max_epoch, verbose=0)

    """

    def __init__(self, n_inputs,
                       activacion="step",
                       lr=0.01):

        self.n = n_inputs + 1              # el bias 'b' se considera como una entrada mas
        self.w = np.random.randn(self.n)   # asigna valores a los pesos
        self.act = activacion              # función de activación del perceptron
        self.lr = lr

        # Historiales
        self.errores = []                  # lista de errores en predicciones (un epoch)
        self.lista_errores = []            # historico de errores (para graficar)
        self.lista_pesos = [self.w.copy()] # historico de pesos (para graficar)
        self.lista_loss = []               # valor promedio del error (loss) en cada época
        self.lista_accuracy = []           # porcentaje de aciertos del modelo en cada época
        self.score_  = 0.0                 # último valor calculado de la precisión (accuracy)
        self.epochs_ = 0
        self.errors_ = []                  # historial de errores por época
    # ==========================================================


    def suma_ponderada(self, xi):
        """
        Calcula la suma ponderada: b + w1x1 + w2x2 + ...
        """
        self.X = np.append(1, xi)  # X = [1, x1, x2, ...] vector de entradas aumentado
        return self.w.dot(self.X)
    # ==========================================================


    def activacion_func(self, z: float) -> float:
        """
        Aplica la función de activación configurada
        Para el perceptrón clásico, la función de activación es 100% discreta (step).
        Si se usa sigmoid o tanh, se está entrenando una neurona de regresión logística
        ese caso requiere la regla de aprendizaje del gradiente
        """
        if self.act == "step":
            # Funcion hard tresshold, step o umbral (default)
            return 1.0 if z >= 0 else 0.0

        elif self.act == "relu":
            # Funcion ReLu
            return max(0, z)

        elif self.act == "sigmoid":
            # Funcion logistica (sigmoide)
            return 1. / (1 + np.exp(-z))

        elif self.act == "tanh":
            # Funcion tangente hiperbolica
            return np.tanh(z)

        else:
            raise ValueError(f"Función de activación '{self.act}' no reconocida.")
    # ==========================================================


    def predict(self, xi):
        """
        Propagación hacia adelante: predice la etiqueta de una entrada xi.
        Método que predice la etiqueta para datos no vistos,
        en base a los datos de entrada y los pesos ajustados.
        """
        y_hat = self.activacion_func(self.suma_ponderada(xi))

        # Binariza la salida si es una función continua
        if self.act in ["sigmoid", "tanh"]:
            return 1.0 if y_hat >= 0.5 else 0.0
        return y_hat
    # ==========================================================


    def update_pesos(self, x_train, y_train):
        """
        Recibe UN dato de entrenamiento
        Actualiza los pesos solo si el error es no nulo
        """
        y_pred = self.predict(x_train) # la salida obtenida (predicha) por el perceptron
        error = y_train - y_pred       # diferencia entre la salida correcta y la obtenida

        if error == 0:   return 0      # dato correcto, pasa al siguiente dato

        # Actualización de pesos
        for i in range(self.n):
            # ajusta el valor de los pesos (de todos los xi donde la entrada sea !=0)
            self.w[i] += self.lr * error * self.X[i]  # X incluye X[0]=1, asi actualiza w[0]

        self.lista_pesos.append(self.w.copy())
        return error
    # ==========================================================


    def fit(self, train_data, lr=0.05, max_epoch=2000, verbose=0, resumen=True):
        """
        Método fit, entrena el modelo con los datos de entrenamiento,
        ajustando los pesos en base a los datos de entrada y las etiquetas correspondientes.

        INPUT
        -----
        X : numpy 2D array. Cada fila corresponde a un ejemplo de entrenamiento.
        y : numpy 1D array. Etiqueta (0 ó 1) de cada ejemplo.

        OUTPUT
        ------
        self: El modelo entrenado.
        Retorna array de pesos wi tras el entrenamiento
        """

        self.lr = lr          # modifica valor por defecto
        # Soporta n entradas: asumimos que la última columna es la etiqueta
        X = train_data[:, :self.n - 1] # Extrae X=(x1,x2,...)
        y = train_data[:, self.n - 1]  # Extrae y=(y1,y2,...)
        epoca = 0
        n_err = 1
        self.errors_.clear()

        while epoca < max_epoch:
            self.errores.clear()
            epoca += 1       # cuenta numero de iteraciones
            n_err = 0        # numero de datos mal clasificados

            for dato in train_data:
                x_train = dato[:self.n - 1]  # extrae caracteristicas [x1,x2,..]
                y_train = dato[self.n - 1]   # extrae etiquetas [y] (salida esperada)ada)

                # actualiza pesos (si hay error)
                err = self.update_pesos(x_train, y_train)
                self.errores.append(err)  # añade resultado a la lista de errores
                if err != 0:  n_err += 1  # dato mal clasificado

            # Calcula métricas
            epoch_score = self.score(X, y)
            self.lista_accuracy.append(epoch_score)
            self.errors_.append(n_err)
            mean_loss = np.mean([abs(e) for e in self.errores]) if len(self.errores) else 0.0
            self.lista_loss.append(mean_loss)

            if verbose:
                print(f"Época {epoca:3d} | Errores: {n_err:3d} | "
                      f"Score: {epoch_score:.3f} | Loss: {mean_loss:.3f}")

            if n_err == 0:
                if verbose:
                    print(f"Entrenamiento completado en {epoca} iteraciones ✅")
                break

        self.epochs_ = epoca

        if n_err != 0:
            print(f"-------------------------------------------------")
            print(f"Entrenamiento NO convergió tras {max_epoch} épocas.")
            print(f"Revisa si los datos son linealmente separables o ajusta lr={self.lr}")

        if resumen:
            self.mostrar_resumen(X, y)

        return self.lista_pesos
    # ==========================================================


    @staticmethod
    def loss(y_pred, y_true):
        """
        Error cuadrático medio.
        """
        return np.mean((y_pred - y_true) ** 2)
    # ==========================================================

    @staticmethod
    def accuracy(y_pred, y_true):
        """
        Exactitud del modelo.
        """
        correct = np.sum(y_pred == y_true)
        return correct / len(y_true)
    # ==========================================================


    def score(self, X, y):
        """
        Calcula la proporción de aciertos del modelo.
        """
        predictions = np.array([self.predict(xi) for xi in X])
        self.score_ = round(self.accuracy(predictions, y), 4)
        return self.score_
    # ==========================================================


    def mostrar_resumen(self, X, y):
        """
        Muestra un resumen del estado del modelo tras el entrenamiento.
        """
        print("\n================ RESULTADOS DEL ENTRENAMIENTO ================")
        print(f"  Épocas ejecutadas: {self.epochs_}")
        print(f"  Tasa de aprendizaje (lr): {self.lr}")
        print(f"  Precisión final (score): {self.score_:.4f}")
        print(f"  Último error promedio:   {self.lista_loss[-1]:.4f}")
        print(f"  Pesos finales: {np.round(self.w, 4)}")
        print("==============================================================\n")
######################################################################

### <u>¿Cómo se usa?</u>: Ejemplos de uso

 2.1- Creamos un perceptron de 2 entradas


In [ ]:
# creamos la instancia, automaticamente asigna pesos iniciales (aleatorios)
perceptron_2entradas = Perceptron(2)

print("Pesos w=(w0, w1, w2)=", perceptron_2entradas.w)

 2.2- Propagacion feed-forward

In [ ]:
# Predice la etiqueta de una entrada concreta
x=[0,1]
y_hat = perceptron_2entradas.predict(x)

print("Prediccion para x=", x," ->  y_hat=", y_hat)

2.3- Actualizacion

In [ ]:
# Nuevo dato de entrenamiento
input_data = np.array([ [0,1, 0] ])

# Entrena (actualiza) el perceptron
perceptron_2entradas.fit(input_data )

print("Pesos w=", perceptron_2entradas.w)

In [ ]:
# importamos los paquetes necesarios

#import numpy as np
import matplotlib.pyplot as plt

<hr>

## <b>3 - Funciones auxiliares</b>
Son funciones complementarias que ayudan a representar graficamente los resultados:
 - normaliza(X)
 - recta_decision(wi)
 - plot_2D(input_data, pesos, labels)

In [ ]:
def normaliza(X: np.ndarray):
    '''
    Reescala entre 0 y 1 los datos de entrada x1, x2,...,
    El valor minimo va a ser 0 y el maximo 1 (para cada coordenada x)
    Asi todos los puntos quedan dentro de un cuadrado de 1x1

    INPUT
    ------
    Datos de entrenamiento:  X = np.array([[x1, x2, x3,...],
                                           [  ,   ,   ,...],
                                           ...
                                          ])
    OUPUT
    -----
    Retorna un numpy array con los valores reescalados
    '''
    return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
##############################


def normaliza_lims(X):

    return X.min(axis=0), X.max(axis=0)
##############################



def recta_decision(wi):
    '''
    Obtiene la recta que separa las 2 clases a partir de los pesos
    Entrada: wi = [b, w1, w2]  # pesos ajustados
    Salidas:  m, n
    '''

    #print("Pesos ajustados", wi)  # muestra los pesos ajustados
    b = wi[0]
    w1= wi[1]
    w2= wi[2]

    # Como hemos entrenado nuestro modelo con dos características x1 y x2,
    # nuestra línea de decisión viene determinada por: w1x1 + w2x2 + b = 0
    # por tanto:  x2 = -(w1/w2)x1 - (b/w2)

    print("\n>>> Linea decision:")
    print('------------------')
    print(f" b = {round(b,2)}\n w1= {round(w1,2)}\n w2= {round(w2,2)}")
    m =-(w1/w2)
    n = -b/w2
    print(f" m={round(m,2)}, n={round(n,2)}")

    return m, n
####################################



def plot_2D(input_data, wi, labels):
    '''
    Dibuja recta que separa las 2 clases (y=0, y!=0)
    INPUT
    -----
    input_data = np.array([[x1, x2, y],
                           [  ,   ,   ,...],
                           ...
                          ])
    wi = [b, w1, w2]  <--  pesos ajustados
    labels: etiquetas para graficar
    '''
    titulo =  labels[0]
    label_x = labels[1][0]
    label_y = labels[1][1]
    label_clase0= labels[2][0]
    label_clase1= labels[2][1]

    # hallamos los valores extremos (x_min, y_min) (x_max, y_max)
    x_min = input_data[0][0] # tomamos x1 del primer dato: input_data[0]
    y_min = input_data[0][1] # tomamos x2 del primer dato
    x_max = x_min
    y_max = y_min

    for x1,x2,y in input_data:
        if x_min>x1: x_min=x1
        if x_max<x1: x_max=x1
        if y_min>x2: y_min=x2
        if y_max<x2: y_max=x2

    plt.figure(figsize=(6, 6))
    #print(f"[x_min,x_max] = [{x_min}, {x_max}]")
    margenX =0.1*(x_max-x_min)
    margenY =0.1*(y_max-y_min)

    x_min = x_min - margenX
    x_max = x_max + margenX
    y_min = y_min - margenY
    y_max = y_max + margenY

    # limites grafica
    plt.xlim([x_min, x_max])
    plt.ylim([y_min, y_max])
    #plt.ylim(.0, .9)

    # variables para graficar (alturas y pesos)

    x1_clase0 =[] # coord_x punto azul
    x2_clase0 =[] # coord_y punto azul
    x1_clase1 =[] # coord_x punto rojo
    x2_clase1 =[] # coord_y punto rojo

    # agrupamos los valores de las x y las y en listas separadas

    for x1,x2,clase in input_data:

        if clase==0:
            x1_clase0.append(x1)
            x2_clase0.append(x2)
        if clase==1:
            x1_clase1.append(x1)
            x2_clase1.append(x2)
    '''
    print("Clase 0:")
    print("x1",x1_clase0)
    print("x2",x2_clase0)
    print("Clase 1:")
    print("x1",x1_clase1)
    print("x2",x2_clase1)
    '''
    plt.plot(x1_clase1, x2_clase1, 'or', label=label_clase1)   # Puntos rojos
    plt.plot(x1_clase0, x2_clase0, 'ob', label=label_clase0)   # Puntos azules

    # Dibujamos la linea de decision
    m, n = recta_decision(wi)

    plt.plot([x_min, x_max],
             [m*x_min+n, m*x_max+n],
             '-m',
             label='Linea de decisión')

    plt.title(titulo)
    plt.legend()
    plt.xlabel(label_x) # Eje x1
    plt.ylabel(label_y) # Eje x2
    plt.show()
#####################

<hr>


## <b> 4 - Ejemplos completos</b>




### 4.-Ej1:  <u>Aprendizaje funciones lógicas</u>
Veamos si el perceptron puede entrenarse para verificar funciones lógicas


<table>
<tr>
<td  style="width: 20%; text-align: center;">    
  y = x<sub>1</sub><strong> OR </strong>x<sub>2</sub>

 <table style="border-collapse: collapse; border-style: solid; margin-left: auto; margin-right: auto; color: #5e9ca0;" border="1" cellspacing="2">   
<tbody>
<tr style="background-color: #5e9ca0; color:#333">
<td><strong>x<sub>1</sub></strong></td>
<td><strong>x<sub>2</sub></strong></td>
<td><strong>y</strong></td>
</tr>
<tr>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>0</td>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>1</td>
<td>0</td>
<td>1</td>
</tr>
<tr>
<td>1</td>
<td>1</td>
<td>1</td>
</tr>
</tbody>
</table>
    
</td>
<td  style="width: 20%; text-align: center;">   
  y = x<sub>1</sub><strong> AND </strong>x<sub>2</sub>
<table style="border-collapse: collapse; border-style: solid; margin-left: auto; margin-right: auto; color: #5e9ca0;" border="1" cellspacing="2">
<tbody>
<tr style="background-color: #5e9ca0; color:#333">
<td><strong>x<sub>1</sub></strong></td>
<td><strong>x<sub>2</sub></strong></td>
<td><strong>y</strong></td>
</tr>
<tr>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>0</td>
<td>1</td>
<td>0</td>
</tr>
<tr>
<td>1</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>1</td>
<td>1</td>
<td>1</td>
</tr>
</tbody>
</table>
     
</td>
<td  style="width: 20%; text-align: center;">   
  y = x<sub>1</sub><strong> XOR </strong>x<sub>2</sub>
    
<table style="border-collapse: collapse; border-style: solid; margin-left: auto; margin-right: auto;color: #5e9ca0;" border="1" cellspacing="2">
<tbody>
<tr style="background-color: #5e9ca0; color:#333">
<td><strong>x<sub>1</sub></strong></td>
<td><strong>x<sub>2</sub></strong></td>
<td><strong>y</strong></td>
</tr>
<tr>
<td>0</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>0</td>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>1</td>
<td>0</td>
<td>1</td>
</tr>
<tr>
<td>1</td>
<td>1</td>
<td>0</td>
</tr>
</tbody>
</table>
    
</td>   
 </tr>
</table>   
    
    


In [ ]:
# Funcion OR        #[x1,x2, y]
data_or = np.array([
                     [0,0, 0],
                     [0,1, 1],
                     [1,0, 1],
                     [1,1, 1]  # La salida y es 1 cuando alguna entrada es 1
                    ])

labels = np.array([ "Puerta OR",
                   ["x1", "x2"],         # X=[x1, x2]
                   ['clase0','clase1']   # y=[clase0, clase1]
                  ], dtype=object)
###################################

# Creamos un perceptron de 2 entradas
perceptron_or = Perceptron(2)

# lo entrenamos con los valores de la tabla de verdad de una funcion OR
wi = perceptron_or.fit(data_or, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(data_or, wi[-1], labels)


In [ ]:
# Funcion AND        #[x1,x2, y]
data_and = np.array([
                      [0,0, 0],
                      [0,1, 0],
                      [1,0, 0],
                      [1,1, 1]  # La salida y es 1 cuando todas las entradas son 1
                     ])

labels = np.array([ "Puerta AND",
                   ["x1", "x2"],         # X=[x1, x2]
                   ['clase0','clase1']   # y=[clase0, clase1]
                  ], dtype=object)
###################################

# Creamos un perceptron de 2 entradas
perceptron_and = Perceptron(2)

# lo entrenamos con los valores de la tabla de verdad de una funcion AND
wi = perceptron_and.fit(data_and, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(data_and, wi[-1], labels)

In [ ]:
# funcion XOR         #[x1,x2, y]
data_xor =  np.array([
                      [0,0, 0],
                      [0,1, 1],
                      [1,0, 1],
                      [1,1, 0]  # La salida es 0 cuando las 2 entradas son iguales
                     ])

labels = np.array([ "Puerta XOR",
                   ["x1", "x2"],         # X=[x1, x2]
                   ['clase0','clase1']   # y=[clase0, clase1]
                  ], dtype=object)
###################################

# Creamos un perceptron de 2 entradas
perceptron_xor = Perceptron(2)

# lo entrenamos con los valores de la tabla de verdad de una funcion XOR
wi = perceptron_xor.fit(data_xor, max_epoch=50, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(data_xor, wi[-1], labels)

In [ ]:
# En este caso graficamos el historico de pesos, para ver que no convergen

# Convertimos historial a array y graficamos
W = np.array(perceptron_xor.lista_pesos)

plt.figure(figsize=(8,4))
for i in range(W.shape[1]):
    plt.plot(W[:, i], label=f"w{i}")

plt.xlabel("Actualizaciones de peso")
plt.ylabel("Valor del peso")
plt.title("Histórico de Pesos (clase completa)")
plt.legend()
plt.show()

# la funcion XOR no es linealmente separable
# Un Perceptron NO PUEDE representar un puerta XOR

##### Ejercicios a completar

In [ ]:
# Funcion NAND
input_data = np.array([#[x1,x2, y]
                      [0,0, 1],
                      [0,1, 1],
                      [1,0, 1],
                      [1,1, 0]  # La salida y es 0 si todas las entradas son 1
                     ])

labels = np.array([ "Puerta NAND",
                   ["x1", "x2"],         # X=[x1, x2]
                   ['clase0','clase1']   # y=[clase0, clase1]
                  ], dtype=object)
###################################


In [ ]:
# Funcion NOT
input_data = np.array([#[x1,x2, y]
                      [0,0, 1],
                      [0,1, 1],
                      [1,0, 0],
                      [1,1, 0]  # La salida y es la inversa de x1
                     ])

labels = np.array([ "Puerta NOT",
                   ["x1", "x2"],         # X=[x1, x2]
                   ['clase0','clase1']   # y=[clase0, clase1]
                  ], dtype=object)

###################################

### 4.-Ej2:  <u>Ejemplo genérico</u>
Usando el perceptron, entrenado con un conjunto de datos dado, obtenemos la recta de decisión que nos permite realizar predicciones.

Partimos de un conjunto de datos 'X' junto a sus clases 'y'

In [ ]:
# conjunto de datos: 14 observaciones -> X=[x1,x2]
X = [
     [8,7], [4,10], [9,7], [7,10], [9,6], [4,8], [10,10],
     [2,7], [8,3],  [7,5], [4,4],  [4,6], [1,3], [2,5]
    ]

# vector de clasificacion y (features)
y = [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

# ampliamos el vector de datos (caracteristicas) con el vector de clases (etiquetas)
input_data = np.array(X)
input_data = np.insert(input_data, 2, y, axis=1)

labels = np.array([ "Recta que separa las clases 0 y 1",
                   ["x1", "x2"],          # X=[x1, x2]
                   ['Clase 0','Clase 1']  # y=[clase0, clase1]
                  ], dtype=object)

###################################

# Creamos un perceptron de 2 entradas
perceptron_ej2 = Perceptron(2)

# lo entrenamos con los datos contenidos en 'input_data'
wi = perceptron_ej2.fit(input_data, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(input_data, wi[-1], labels)

####  Evolución de los métodos loss() y accuracy()

 Muestra cómo la pérdida y la precisión, (loss y accuracy), mejora (o no) mientras aprende.

 Una vez entrenado el perceptron, $fit()$, se guardan en 'lista_loss' y
 en 'lista_accuracy' los valores por cada epoca, y los mostramos en la gráfica


In [ ]:
##### Gráficas de loss y Accuracy por epoca #####

  plt.figure(figsize=(12, 3))
  plt.subplot(1, 2, 1)
  plt.plot(perceptron_ej2.lista_loss, '-o')
  plt.title('Loss por época')
  plt.xlabel('Epoca')
  plt.ylabel('Loss (media abs error)')
  plt.ylim(-0.05, 1.05)   # margen visual
  plt.grid(True)

  plt.subplot(1, 2, 2)
  plt.plot(perceptron_ej2.lista_accuracy, '-o')
  plt.title('Accuracy por época')
  plt.xlabel('Epoca')
  plt.ylabel('Accuracy')
  plt.ylim(-0.05, 1.05) # margen visual
  plt.grid(True)
  plt.show()

In [ ]:
# Ejemplo 3.3
# -----------

# Datos de 10 personas -> [edad, ahorro(1000$)]
personas = np.array([
                     [25, 32], [40, 38], [30, 20.9], [44, 10],[35, 51], [50, 61],
                     [44, 151], [36, 89], [50, 261], [37, 165], [58, 352]
                    ])

# 0 : denegada,    1 : aprobada
clases = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

input_data = np.array(personas)                        # convertimos a numpy array
input_data = np.insert(input_data, 2, clases, axis=1)  # añade vector de clases -> [x1, x2, y]


labels = np.array([ "Prestamos a clientes",
                   ["Edad", "Ahorro (1000€)"], # x1 , x2
                   ['Denegado','Concedido']    # y_clase0, y_clase1
                  ], dtype=object)

###################################

# Creamos un perceptron de 2 entradas
perceptron_ej3 = Perceptron(2)

# lo entrenamos con los datos contenidos en 'input_data'
wi = perceptron_ej3.fit(input_data, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(input_data, wi[-1], labels)

In [ ]:
print(perceptron_ej3.errors_)
plt.plot(range(1, len(perceptron3.errors_) + 1), perceptron3.errors_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Number of updates')

# plt.savefig('images/02_07.png', dpi=300)
plt.show()

In [ ]:
### Ejemplo de prediccion

edad = float(input("Introduce tu edad (años): "))
money = float(input("Introduce tu dinero ahorrado (€): "))/1000


if perceptron3.predict([edad,money]) == 1:
    resultado ="Prestamo concedido"
else:
    resultado ="Prestamo no concedido"

print("Respuesta de la red neuronal...", resultado)

### 4.-Ej3: <u>Clasificación  de hombres y mujeres por su peso y altura</u>
Disponemos de unos datos de pesos y alturas de hombre y mujeres.
A partir de estos datos podemos realizar predicciones del genero al que pertenece un persona a partir de su peso y altura.

In [ ]:
# Ejemplo clasificacion por peso y altura
# -----------

# Datos de pesos y alturas de hombres y mujeres adultos
input_data= np.array([[56, 1.70, 1], # Mujer de 1.70m y 56kg
                      [63, 1.72, 0], # Hombre de 1.72m y 63kg
                      [50, 1.60, 1], # Mujer de 1.60m y 50kg
                      [63, 1.70, 0], # Hombre de 1.70m y 63kg
                      [66, 1.74, 0], # Hombre de 1.74m y 66kg
                      [55, 1.58, 1], # Mujer de 1.58m y 55kg
                      [62, 1.66, 1],
                      [80, 1.83, 0], # Hombre de 1.83m y 80kg
                      [70, 1.82, 0], # Hombre de 1.82m y 70kg
                      [54, 1.65, 1]  # Mujer de 1.65m y 54kg
                     ])

labels=np.array([ "Recta que 'separa' hombres y mujeres",
                 ["Peso (Kg)", "Altura (m)"], # x1 y x2
                 ['Hombres','Mujeres']        # y_clase0 =hombre, y_clase1 =mujer
                ], dtype=object)

###################################

# Creamos un perceptron de 2 entradas
perceptron_ej3 = Perceptron(2)

# lo entrenamos con los datos contenidos en 'input_data'
wi = perceptron_ej3.fit(input_data, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(input_data, wi[-1], labels)


En el caso anterior es muy posible que los pesos no hayan convergido, o si lo hecho haya sido a costa de un gran numero de iteraciones.
Esto se debe a cómo estan distribuidos los puntos en el espacio x1:x2
Se puede evitar normalizando los datos entre 0 y 1.
Lo vemos en el siguiente ejemplo:

In [ ]:
# Ejemplo 3 (bis)
###################################

# Creamos de nuevo un perceptron de 2 entradas
perceptron_ej3b = Perceptron(2)

# Normalizamos los datos (mejora la convergencia, reduce epochs)
input_data_norm = normaliza(input_data)

# lo entrenamos con los datos contenidos en 'input_data'
wi = perceptron_ej3b.fit(input_data_norm, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(input_data_norm, wi[-1], labels)

Ahora con el perceptrón entrenado podemos realizar <b>predicciones</b>:

In [ ]:
### Ejemplo de prediccion (datos normalizados)

peso = float(input("Introduce tu peso (kilogramos): "))
altura = float(input("Introduce tu estatura (centimetros): "))/100

# hay que normalizar los datos de entrada,
# para que tenga la misma escala que los datos de entrenamiento
# OJO, altura y peso deben estar dentro del rango [x_min, x_max]
x_min, x_max= normaliza_lims(input_data)
peso_norm = (peso-x_min[0]) / (x_max[0]-x_min[0])
altura_norm = (altura-x_min[1]) / (x_max[1]-x_min[1])


if perceptron_ej3b.predict([peso_norm, altura_norm]) == 1:
    resultado ="mujer"
else:
    resultado ="hombre"

print("La Red Neuronal predice que eres...", resultado)


### 4.-Ej4: <u>Prestamos concedidos a clientes</u>
En este ejemplo, suponemos que una entidad bancaria debe decidir si conceder un préstamo a un cliente. Para ello dispone de datos de otros clientes que puede utilizar para realizar la predicción de si el cliente devolverá el dinero.

In [ ]:
# Ejemplo clasificacion por ingresos y edades
# -----------

# Datos de ingresos y edades de clientes,
# x1=edad, x2=saldo cuenta, y=1 representa todos los prestamos devueltos
input_data= np.array([[26, 6e5,0],
                      [21, 2500, 0], # moroso
                      [32, 1e5, 0],  # moroso
                      [36, 1.2e6, 1],
                      [43, 4.2e5, 1],
                      [56, 2.70e5, 1]

                     ])

labels=np.array([ "Prestamos devueltos",
                 ["Edad (años)", "Saldo (€)"], # x1 y x2
                 ['Moroso','Solvente']        # y_clase0 =moroso, y_clase1 =solvente
                ], dtype=object)

###################################

# Creamos un perceptron de 2 entradas
perceptron_ej4 = Perceptron(2)

# Normalizamos los datos (mejora la convergencia, reduce epochs)
input_data_norm = normaliza(input_data)

# lo entrenamos con los datos contenidos en 'input_data'
wi = perceptron_ej4.fit(input_data_norm, verbose=0)
# wi es el historico de pesos, el ultimo wi[-1] son los pesos ajustados

# Graficamos los puntos y la linea de decision
plot_2D(input_data_norm, wi[-1], labels)

In [ ]:
### Ejemplo de prediccion (datos normalizados)

edad = float(input("Introduce la edad: "))
saldo = float(input("Introduce el saldo en Cuenta Corriente (miles): "))/1000

# hay que normalizar los datos de entrada,
# para que tenga la misma escala que los datos de entrenamiento
# OJO, edad y saldo deben estar dentro del rango [x_min, x_max]
x_min, x_max= normaliza_lims(input_data)
edad_norm   = (edad-x_min[0]) / (x_max[0]-x_min[0])
saldo_norm  = (saldo-x_min[1]) / (x_max[1]-x_min[1])


if perceptron_ej4.predict([edad_norm, saldo_norm]) == 1:
    resultado ="concedido!"
else:
    resultado ="denegado!"

print("El préstamo ha sido...", resultado)